# Longitude, Latitude Converter

In this script, we use the requests library to send a GET request to the OpenStreetMap Nominatim API. We pass the address as a query parameter and limit the results to one match. We also specify the response format as JSON.

Once we receive the JSON response, we extract the latitude and longitude from the first match, if one exists. If there are no matches, we leave the latitude and longitude fields empty.

Finally, we write the address, latitude, and longitude to the output CSV file.

Note that OpenStreetMap Nominatim has usage limits, so you should read the usage policy and make sure that your use of the service complies with the terms of use.

This process using openstreetmap did work for converting the addresses to longitude and latitude. It has in its user agreement not to do big batch dataprocessing. I'm not sure if one semester is big batch. In this case, it was about 6100 addresses. It only allows one request a second. Therefore, it took one hour and fourty-two minutes to download, but to my surprise, it worked! You'll notice, *nominatim* is the extension of the library *geopy* that is used in the call geopy.geocoders.Nominatim(user_agent = 'MyCoder')

In [ ]:
import csv
import requests

# OpenStreetMap Nominatim API URL
NOMINATIM_URL = 'https://nominatim.openstreetmap.org/search'

# CSV file containing addresses
input_file = '202310 Addresses Missing.csv'

# CSV file to write output
output_file = 'long_lat_202310_missing.csv'

# Open input file and output file
with open(input_file, 'r') as csv_input, open(output_file, 'w', newline='') as csv_output:
    # Create CSV reader and writer objects
    reader = csv.reader(csv_input)
    writer = csv.writer(csv_output)

    # Write header row to output file
    writer.writerow(['Address', 'Latitude', 'Longitude'])

    # Loop through each row in the input file
    for row in reader:
        # Get the address from the current row
        address = row[0]

        # Send a GET request to the OpenStreetMap Nominatim API
        response = requests.get(NOMINATIM_URL, params={'q': address, 'format': 'json', 'limit': 1})

        # Parse the JSON response
        json_data = response.json()

        # Get the latitude and longitude from the JSON response
        if len(json_data) > 0:
            latitude = json_data[0]['lat']
            longitude = json_data[0]['lon']
        else:
            latitude = ''
            longitude = ''

        # Write the address, latitude, and longitude to the output file
        writer.writerow([address, latitude, longitude])
        

## The Code Below Is For Accessing Mapquest Geocode API

Mapquest did an outstanding job. You're allowed 15,000 requests a month. So that's roughly two semester's worth, but it did find every single address and came back with the correct longitude and latitude. It is limited, mostly, to the United States. The format for the address file is, no headers, just a matrix, with the first column as the street address, the second column is the city, the third column is the state, the fourth is the zip code. It does generally only load one row per second so 6100 takes an hour and 20 to an hour and 40 minutes. I did one load at night and it ran it a little faster. During the day, expect a semester of 6500 students to take two to three hours. 

In [3]:
import csv
import requests
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access environment variables
API_key = os.getenv('API_KEY')


In [ ]:
# MapQuest Geocoding API URL
MAPQUEST_URL = 'https://www.mapquestapi.com/geocoding/v1/address'

# API key
API_KEY = API_key

# CSV file containing addresses
input_file = '202310 Addresses.csv'

# CSV file to write output
output_file = 'long_lat_202310.csv'

# Open input file and output file
with open(input_file, 'r') as csv_input, open(output_file, 'w', newline='') as csv_output:
    # Create CSV reader and writer objects
    reader = csv.reader(csv_input)
    writer = csv.writer(csv_output)

    # Write header row to output file
    writer.writerow(['address', 'longitude', 'latitude'])

    # Loop through each row in the input file
    for row in reader:
        # Get the address components from the current row
        street, city, state, zip_code = row

        # Build the address string
        address = f'{street}, {city}, {state} {zip_code}'

        # Send a GET request to the MapQuest Geocoding API
        response = requests.get(MAPQUEST_URL, params={'key': API_KEY, 'location': address})

        # Parse the JSON response
        json_data = response.json()

        # Get the latitude and longitude from the JSON response
        if json_data['results']:
            latitude = json_data['results'][0]['locations'][0]['latLng']['lat']
            longitude = json_data['results'][0]['locations'][0]['latLng']['lng']
        else:
            latitude = ''
            longitude = ''

        # Write the address, latitude, and longitude to the output file
        writer.writerow([address, longitude, latitude])